# Prototyping the new json schema


As part of the consolidation of the evidence objects in the backend, we are re-modeling the [json schema](https://github.com/opentargets/json_schema) to reflect the new simplified/flattened design.

**Link to ticket: [#1249](https://github.com/opentargets/platform/issues/1249)**

Based on the meeting we had on 2020.11.11 the following conclusions were reached:

* We need to maintain a json schema that guides our data providers and can be used as template to generate evidence strings.
* The schema will reflect the concepts of the new platform design, so the units of the schema is going to be data source centric instead of data type.
* Each of the valuable columns will be defined in a common section.
* For each data source there will only be a list of required fields.
* We haven't reached a consensus on how the unique association fields are defined, and at which point of the evidence generation. So for the first iteration of the json schema, the unique_association_fields will be omitted.

The schema is written based on the most recent iteration of the [evidence schema review](https://docs.google.com/spreadsheets/d/11jdPCo_vxY3jaP54xKTsXBshR5HMrpUf5oXJNgtbKm8/edit#gid=1735847104) document.

The technical approach:

* To avoid manual work with the json document, I'm collating information in an excel file and will use that as a source for the definitions.
* The same excel file will be used to get the source names from where we are expecting the given field.

## The first run completed:

- [X] processing the review document to get the rough list of fields
- [x] get fields2datasource mapping
- [x] generate json schema based on the meeting


## The first run didn't cover:

- [ ] some fields are missing eg. uniprot id
- [ ] some fields shoudl not be here: `score` and `id`
- [ ] the precise requirements of the fields are still sparse -> add more data to `field_description.xlsx`
- [ ] no structure whatsoever.

## 1. Getting the list of data source for every field

This information is extracted from the evidence schema review file. The end of the process is a comma separated list of data sources for every field. This column is used later to generate the mandatory list of fields for every data source.

In [85]:
# 1. Get the source names for every field:
import pandas as pd
import json
from collections import OrderedDict, defaultdict
import numpy as np
import requests

notnull_table = pd.read_csv('iter9_notnull_table.tsv', sep='\t')
notnull_table.index = notnull_table.key.tolist()
notnull_table.drop('key', axis=1, inplace=True)
notnull_table.head()

,cancer_gene_census,chembl,clingen,crispr,europepmc,eva,eva_somatic,expression_atlas,gene2phenotype,genomics_england,intogen,ot_genetics_portal,phenodigm,phewas_catalog,progeny,reactome,slapenrich,sysbio,uniprot_literature,uniprot_somatic
allelicRequirement,0,0,1075,0,0,0,0,0,2451,0,0,0,0,0,0,0,0,0,0,0
biologicalModelAllelicComposition,0,0,0,0,0,0,0,0,0,0,0,0,564310,0,0,0,0,0,0,0
biologicalModelGeneticBackground,0,0,0,0,0,0,0,0,0,0,0,0,564310,0,0,0,0,0,0,0
clinicalPhase,0,427943,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
clinicalSignificance,0,0,0,0,0,107532,8173,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# Looping through all fields and extracting all sources where the data is present:
lookup = lambda x: ','.join(notnull_table.loc[x].where(lambda x: x != 0).dropna().index.to_list())
pd.Series(notnull_table.index).apply(lookup)


0         clingen,gene2phenotype
1                      phenodigm
2                      phenodigm
3                         chembl
4                eva,eva_somatic
                 ...            
57    cancer_gene_census,intogen
58                      reactome
59    cancer_gene_census,intogen
60    cancer_gene_census,intogen
61            cancer_gene_census
Length: 62, dtype: object

## Reading field description

1. Read excel file with the field descriptions: from [google sheets](https://docs.google.com/spreadsheets/d/1vHoyIsQDBNmUfq2IUdZDoz3G457V5cqW/edit#gid=613969206)
2. Parse values. 
3. Start building json object.


In [194]:
def get_dataframe():
    '''
    This function fetches the field definitinos from google spreadsheets
    '''
    field_description = 'field_descriptions.tsv'

    # The file from now on is stored on google sheets:
    url = 'https://docs.google.com/spreadsheets/d/1vHoyIsQDBNmUfq2IUdZDoz3G457V5cqW/export?format=tsv&id=1vHoyIsQDBNmUfq2IUdZDoz3G457V5cqW&gid=613969206'
    r = requests.get(url)

    with open(field_description, 'w') as f:
        f.write(r.text)

    fields_df = pd.read_csv(field_description, sep='\t')
    return fields_df


def parse_data_sources(df):
    '''
    Parsing dataframe to get list of fields for each data source:
    
    input: features_df
    output: oneof ordered dictionary
    '''
    
    parsed_sources = defaultdict(list)
    for i, row in df.iterrows():
        for source in row['data_source'].split(','):                
            parsed_sources[source].append(row['field_name'])

    # Each data source then exploded into schemas:
    source_schemas = []
    sorted_items = sorted(parsed_sources.items())
    for source, fields in sorted_items:
        source_schema = OrderedDict()

        # Adding property definitions:
        source_schema['properties'] = OrderedDict({'datasourceId': {"const": source}})
        
        for field in fields:
            if field == 'datasourceId':
                continue
                
            source_schema['properties'][field] = {"$ref": f"#/definitions/{field}"}
        
        source_schema['required'] = []
        source_schema["additionalItems"] = False

        # Adding source schema:
        source_schemas.append(source_schema)
        
    return(source_schemas)


def add_definition(row, fields_df):
    '''
    This is the main function to add definitions to the schema
    '''
    
    # If the feature is simple:
    if row['type'] in ['string', 'integer', 'number']:
        return add_simple_definition(row)
    
    # If the feature is complex:
    elif row['type'] == 'array':
        print(row["field_name"])
        return add_array(row, fields_df)
        print(f'complex objectfound: {row["field_name"]}')
    
    
def add_array(row, df):
    '''
    If the definition is an array, things have to be treated separately
    '''
    field_annotation = OrderedDict({'type':'array'})
    field_name = row['field_name']
    
    # If there's a description:
    if isinstance(row['description'], str):
        field_annotation['description'] = row['description']

    # Items are objects: 
    if len(df.loc[df.location == field_name]) > 0:
        field_annotation['items'] = OrderedDict({
            'type': "object",
            'properties': OrderedDict()
        })
        
        sub_df = df.loc[df.location == field_name]
        for index, sub_row in sub_df.iterrows():
             field_annotation['items']['properties'][sub_row['field_name']] = add_definition(sub_row, sub_df)

        
    # Items are string:
    else:
        row['type'] = 'string'
        row['nullable'] = False

        field_annotation['items'] = add_simple_definition(row)
        
    field_annotation["uniqueItems"] = True
    
    return field_annotation
    
    
def add_simple_definition(row):
    '''
    If the feature is a simple object handled easy.
    '''
    field = row['field_name']

    field_annotation = OrderedDict()
    
    # Setting type - maybe nullable:
    if row['nullable']:
        field_annotation['type'] = [row['type'], "null"]
    else:
        field_annotation['type'] = row['type']
    
    # Adding description:
    if isinstance(row['description'], str):
        field_annotation['description'] = row['description']

    # Adding minimum:
    if not np.isnan(row['minimum']):
        field_annotation['minimum'] = row['minimum']
        
    # Adding maximum:
    if not np.isnan(row['maximum']):
        field_annotation['maximum'] = row['maximum']

    # Is it exclusive minimum:
    if not np.isnan(row['exclusiveMinimum']):
        field_annotation['exclusiveMinimum'] = row['exclusiveMinimum']
        
    # Adding pattern:
    if isinstance(row['pattern'], str):
        field_annotation['pattern'] = row['pattern']
        
    # Is there a list of accepted values (might be a list of floats!!):
    if isinstance(row['accepted_values'], str):
        enum_values = row['accepted_values'].split(';')
        try:
            field_annotation['enum'] = [float(x) for x in enum_values]
        except:
            field_annotation['enum'] = enum_values
        
    return field_annotation



In [195]:
# Reloading possible modifications from the xlsx file:
fields_df = get_dataframe()

##
## initialize json schema:
##
schema_obj = OrderedDict({
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "OpenTargets",
    "description": "OpenTargets evidence objects",
    "type": "object",
    "oneOf": {},
    "definitions": OrderedDict()
})

# Adding descxription for all data sources:
schema_obj["oneOf"] = parse_data_sources(fields_df.loc[fields_df.location == 'root'])

##
## Generating field definitions:
##
    
# This will be updated with every row:
root_object = OrderedDict() 
fields_df = get_dataframe()

# Looping through all fields that are in the root of the document:
for i, row in fields_df.loc[fields_df.location == 'root'].iterrows():
    schema_obj['definitions'][row['field_name']] = add_definition(row, fields_df)

# Saving object into json file:
with open('json_schema_v0.2.json', 'w') as f:
    json.dump(schema_obj, f, indent=2)
    
    

biologicalModel
clinicalSignificance
clinicalUrls
diseaseModelAssociatedHumanPhenotypes
diseaseModelAssociatedModelPhenotypes
literature
significantDriverMethods
textMiningSentences
variations


In [193]:
fields_df.loc[fields_df.location == 'root'].sort_values(['field_name'], axis=0)

,field_name,type,location,description,minimum,exclusiveMinimum,maximum,accepted_values,nullable,pattern,data_source
0,allelicRequirement,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"clingen,gene2phenotype"
1,biologicalModel,array,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,phenodigm
4,clinicalPhase,integer,root,NaN,1.0,NaN,4.0,NaN,NaN,NaN,chembl
5,clinicalSignificance,array,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"eva,eva_somatic"
6,clinicalStatus,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chembl
7,clinicalUrls,array,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chembl
10,cohortDescription,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intogen
11,cohortId,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intogen
12,cohortShortName,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intogen
13,confidence,string,root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"clingen,gene2phenotype"
